# Table 7: Eyeriss v1 AlexNet Reproduction

Reproduces Table 7 from micro22-sparseloop-artifact using AccelForge.

**Architecture:** Eyeriss v1 — 168 PEs (14×12), row-stationary dataflow
- DRAM → shared_glb (14 PEColumns) → DummyBuffer Toll (12 PEs) → ifmap/weights/psum spads → MACs

**Workload:** AlexNet conv1-5 with per-layer sparsity densities

**Sparse configs:**
- Conv1: `dense_iact_opt` — Outputs UOP+RLE at DRAM only
- Conv2-5: `sparse_iact_opt` — Inputs+Outputs UOP+RLE at DRAM, gating at weights_spad

In [ ]:
import os
import sys
import pandas as pd

REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, REPO_ROOT)

from accelforge.frontend.spec import Spec
from accelforge.model.main import evaluate_mapping

TABLE7_DIR = os.path.join(REPO_ROOT, 'tests', 'input_files', 'table7')
print(f'Using configs from: {TABLE7_DIR}')

## 1. Sparseloop Reference Data

Reference values from `table7_eyeriss_setup/ref_outputs/`

In [ ]:
# Sparseloop reference (sparse case)
SL_REF = {
    'conv1': {'cycles': 2_838_528, 'energy_uJ': 2_059.86, 'actual_computes': 437_133_312,
              'dense_computes': 437_133_312, 'sparse_config': 'sparse_dense_iact.yaml'},
    'conv2': {'cycles': 4_128_768, 'energy_uJ': 3_160.50, 'actual_computes': 578_027_520,
              'dense_computes': 963_379_200, 'sparse_config': 'sparse_sparse_iact.yaml'},
    'conv3': {'cycles': 1_916_929, 'energy_uJ': 1_534.63, 'actual_computes': 164_472_423,
              'dense_computes': 598_081_536, 'sparse_config': 'sparse_sparse_iact.yaml'},
    'conv4': {'cycles': 1_437_697, 'energy_uJ': 1_110.05, 'actual_computes': 92_852_159,
              'dense_computes': 448_561_152, 'sparse_config': 'sparse_sparse_iact.yaml'},
    'conv5': {'cycles':   958_464, 'energy_uJ':   756.75, 'actual_computes': 68_779_377,
              'dense_computes': 299_040_768, 'sparse_config': 'sparse_sparse_iact.yaml'},
}

ref_df = pd.DataFrame(SL_REF).T
ref_df.index.name = 'Layer'
display(ref_df)

## 2. Run All Layers (Dense + Sparse)

In [ ]:
def run_layer(layer, sparse=True):
    """Run a single layer. Returns (cycles, energy_uJ, computes, result)."""
    files = [
        os.path.join(TABLE7_DIR, 'arch.yaml'),
        os.path.join(TABLE7_DIR, f'workload_{layer}.yaml'),
        os.path.join(TABLE7_DIR, f'mapping_{layer}.yaml'),
    ]
    if sparse:
        files.append(os.path.join(TABLE7_DIR, SL_REF[layer]['sparse_config']))
    
    spec = Spec.from_yaml(*files)
    result = evaluate_mapping(spec)
    
    cycles = float(result.data['Total<SEP>latency'].iloc[0])
    energy = float(result.data['Total<SEP>energy'].iloc[0]) / 1e6  # pJ -> uJ
    computes = float(result.data['Conv<SEP>action<SEP>MACs<SEP>None<SEP>compute'].iloc[0])
    return cycles, energy, computes, result


def get_action(result, component, tensor, action_type):
    """Get action count from result DataFrame."""
    col = f'Conv<SEP>action<SEP>{component}<SEP>{tensor}<SEP>{action_type}'
    if col in result.data.columns:
        return float(result.data[col].iloc[0])
    return 0.0

In [ ]:
# Run all layers: dense and sparse
dense_results = {}
sparse_results = {}

for layer in SL_REF:
    print(f'Running {layer}...')
    dense_results[layer] = run_layer(layer, sparse=False)
    sparse_results[layer] = run_layer(layer, sparse=True)

print('Done!')

## 3. Dense Comparison

In [ ]:
rows = []
for layer in SL_REF:
    cycles, energy, computes, result = dense_results[layer]
    ref = SL_REF[layer]
    rows.append({
        'Layer': layer,
        'AF Dense Computes': f'{computes:,.0f}',
        'SL Dense Computes': f"{ref['dense_computes']:,}",
        'Match': 'Y' if abs(computes - ref['dense_computes']) < 2 else 'N',
        'AF Dense Cycles': f'{cycles:,.0f}',
        'AF Dense Energy (uJ)': f'{energy:.2f}',
    })

dense_df = pd.DataFrame(rows)
display(dense_df)
print('\nNote: Dense cycles = total_computes / utilized_PEs (compute-bound, no memory BW)')

## 4. Sparse Comparison

In [ ]:
rows = []
for layer in SL_REF:
    cycles, energy, computes, result = sparse_results[layer]
    ref = SL_REF[layer]
    rows.append({
        'Layer': layer,
        'AF Computes': f'{computes:,.0f}',
        'SL Computes': f"{ref['actual_computes']:,}",
        'Compute Match': 'Y' if abs(computes - ref['actual_computes']) < 2 else 'N',
        'AF Cycles': f'{cycles:,.0f}',
        'SL Cycles': f"{ref['cycles']:,}",
        'Cycle Ratio': f"{cycles / ref['cycles']:.2f}x",
        'AF Energy (uJ)': f'{energy:.2f}',
        'SL Energy (uJ)': f"{ref['energy_uJ']:.2f}",
        'Energy Ratio': f"{energy / ref['energy_uJ']:.2f}x",
    })

sparse_df = pd.DataFrame(rows)
display(sparse_df)

## 5. Conv1 Detailed Comparison (Dense)

Conv1 is the primary validation target as it has exact cycle match and weights_spad temporal reuse validation.

In [ ]:
_, _, _, conv1_sparse = sparse_results['conv1']

# === Conv1 Sparse: Per-Component Energy vs Sparseloop ===
# Reference values extracted from timeloop-model.stats.txt
SL_CONV1_ENERGY = {
    'MACs':         961_846_283.06,
    'psum_spad':    227_721_022.34,
    'weights_spad': 319_238_379.69,
    'ifmap_spad':    87_918_847.19,
    'DummyBuffer':   0,
    'shared_glb':   70_184_877.29 + 74_391_611.73,  # I + O
    'DRAM':        142_737_408 + 99_348_480 + 76_474_800,  # W + I + O
}

def get_comp_energy(result, comp):
    total = 0
    for col in result.data.columns:
        if f'energy<SEP>{comp}<SEP>' in col:
            total += float(result.data[col].iloc[0])
    return total

print('=== Conv1 Sparse: Per-Component Energy (pJ) ===')
print(f'{"Component":>15} {"AF (uJ)":>10} {"SL (uJ)":>10} {"Ratio":>8} {"Diff (uJ)":>10}')
print('-' * 60)
af_total = sl_total = 0
for comp in ['MACs', 'psum_spad', 'weights_spad', 'ifmap_spad', 'shared_glb', 'DRAM']:
    af_e = get_comp_energy(conv1_sparse, comp) / 1e6
    sl_e = SL_CONV1_ENERGY[comp] / 1e6
    af_total += af_e; sl_total += sl_e
    ratio = f'{af_e/sl_e:.2f}x' if sl_e > 0 else 'n/a'
    print(f'{comp:>15} {af_e:>10.2f} {sl_e:>10.2f} {ratio:>8} {af_e - sl_e:>+10.2f}')
print(f'{"TOTAL":>15} {af_total:>10.2f} {sl_total:>10.2f} {af_total/sl_total:>7.2f}x {af_total-sl_total:>+10.2f}')

# === Conv1: DRAM Action Counts ===
UTILIZED_PES = 154
print('\n=== Conv1 Sparse: DRAM Action Counts (AF=vectors, SL=scalars, block_size=4) ===')
SL_DRAM = {'W_reads': 1_115_136, 'I_reads': 776_160, 'O_reads': 0, 'O_writes': 455_197}
for tensor, action, sl_key in [('Weights','read','W_reads'), ('Inputs','read','I_reads'),
                                 ('Outputs','read','O_reads'), ('Outputs','write','O_writes')]:
    af_vec = get_action(conv1_sparse, 'DRAM', tensor, action)
    af_scalar = af_vec * 4
    sl_scalar = SL_DRAM[sl_key]
    match = 'MATCH' if abs(af_scalar - sl_scalar) < 4 else f'{af_scalar:,.0f} vs {sl_scalar:,}'
    print(f'  {tensor:>8} {action:>5}: AF_vec={af_vec:>12,.0f}  AF_scalar={af_scalar:>12,.0f}  SL={sl_scalar:>12,}  {match}')

# === Conv1: weights_spad Temporal Reuse Validation ===
ws_reads = get_action(conv1_sparse, 'weights_spad', 'Weights', 'read')
ws_writes = get_action(conv1_sparse, 'weights_spad', 'Weights', 'write')
print(f'\n=== Conv1: weights_spad Temporal Reuse ===')
print(f'  reads total: {ws_reads:,.0f} (SL: {2_838_528*154:,})  per PE: {ws_reads/UTILIZED_PES:,.0f} (SL: 2,838,528)')
print(f'  fills total: {ws_writes:,.0f} (SL: {50_688*154:,})  per PE: {ws_writes/UTILIZED_PES:,.0f} (SL: 50,688)')
print(f'  Temporal reuse ratio: {ws_reads/ws_writes:.1f}x (SL: {2_838_528/50_688:.1f}x)')

## 6. Conv3 Detailed Comparison (Sparse)

Conv3 is the key sparse validation target with input gating at weights_spad.

In [ ]:
_, _, _, conv3_sparse = sparse_results['conv3']

# Conv3 Sparseloop reference per-component energy (pJ)
SL_CONV3_ENERGY = {
    'MACs':         361_896_895.95,
    'psum_spad':    314_805_192.70,
    'weights_spad': 132_377_569.83,
    'ifmap_spad':   120_360_343.63,
    'DummyBuffer':  0,
    'shared_glb':   57_059_921.66 + 357_783_479.14,  # I + O
    'DRAM':        113_246_208 + 63_883_032 + 13_214_408,  # W + I + O
}

print('=== Conv3 Sparse: Per-Component Energy ===')
print(f'{"Component":>15} {"AF (uJ)":>10} {"SL (uJ)":>10} {"Ratio":>8} {"Diff (uJ)":>10}')
print('-' * 60)
af_total = sl_total = 0
for comp in ['MACs', 'psum_spad', 'weights_spad', 'ifmap_spad', 'shared_glb', 'DRAM']:
    af_e = get_comp_energy(conv3_sparse, comp) / 1e6
    sl_e = SL_CONV3_ENERGY[comp] / 1e6
    af_total += af_e; sl_total += sl_e
    ratio = f'{af_e/sl_e:.2f}x' if sl_e > 0 else 'n/a'
    print(f'{comp:>15} {af_e:>10.2f} {sl_e:>10.2f} {ratio:>8} {af_e - sl_e:>+10.2f}')
print(f'{"TOTAL":>15} {af_total:>10.2f} {sl_total:>10.2f} {af_total/sl_total:>7.2f}x {af_total-sl_total:>+10.2f}')

# DRAM action counts
print('\n=== Conv3 Sparse: DRAM Action Counts ===')
SL_DRAM3 = {'W_reads': 884_736, 'I_reads': 380_161, 'O_reads': 0, 'O_writes': 78_654}
for tensor, action, sl_key in [('Weights','read','W_reads'), ('Inputs','read','I_reads'),
                                 ('Outputs','read','O_reads'), ('Outputs','write','O_writes')]:
    af_vec = get_action(conv3_sparse, 'DRAM', tensor, action)
    af_scalar = af_vec * 4
    sl_scalar = SL_DRAM3[sl_key]
    match = 'MATCH' if abs(af_scalar - sl_scalar) < 4 else f'{af_scalar:,.0f} vs {sl_scalar:,}'
    print(f'  {tensor:>8} {action:>5}: AF_vec={af_vec:>12,.0f}  AF×4={af_scalar:>12,.0f}  SL={sl_scalar:>12,}  {match}')

# Gating validation
CONV3_PES = 156  # 13 columns × 12 PEs
ws_reads_3 = get_action(conv3_sparse, 'weights_spad', 'Weights', 'read')
gated_computes = get_action(conv3_sparse, 'MACs', 'gated_compute', '')
# Check via column name pattern
for col in conv3_sparse.data.columns:
    if 'gated' in col:
        val = float(conv3_sparse.data[col].iloc[0])
        if val > 0:
            print(f'\n  {col}: {val:,.0f}')

print(f'\n=== Conv3: Gating Validation (156 PEs) ===')
print(f'  weights_spad actual reads: {ws_reads_3:,.0f} (SL: {1_054_311*156:,}, per PE: {ws_reads_3/CONV3_PES:,.0f} vs 1,054,311)')
print(f'  SL gated reads per PE: 2,779,545  →  total: {2_779_545*156:,}')
print(f'  SL algorithmic reads per PE: 3,833,856  →  total: {3_833_856*156:,}')

## 7. Validation Summary

### Cycles Comparison (Sparse)
| Layer | AF Cycles | SL Cycles | Ratio |
|-------|-----------|-----------|-------|
| conv1 | 2,838,528 | 2,838,528 | **1.00x** |
| conv2 | 4,128,768 | 4,128,768 | **1.00x** |
| conv3 | 1,916,928 | 1,916,929 | **1.00x** |
| conv4 | 1,437,696 | 1,437,697 | **1.00x** |
| conv5 | 958,464 | 958,464 | **1.00x** |

### Energy Comparison (Sparse)
| Layer | AF Energy (uJ) | SL Energy (uJ) | Ratio |
|-------|-----------------|-----------------|-------|
| conv1 | 1,960.69 | 2,059.86 | **0.95x** |
| conv2 | 3,045.16 | 3,160.50 | **0.96x** |
| conv3 | 1,517.25 | 1,534.63 | **0.99x** |
| conv4 | 1,039.11 | 1,110.05 | **0.94x** |
| conv5 | 709.65 | 756.75 | **0.94x** |

### Exact Matches
| Metric | Layers | Details |
|--------|--------|---------|
| **Cycles** | All 5 | Within 1 cycle of Sparseloop reference |
| **Dense compute counts** | All 5 | 437M, 963M, 598M, 449M, 299M |
| **Sparse compute counts** | All 5 | Within 1 of Sparseloop reference |
| **DRAM Weights reads** | All 5 | Vector count x 4 = SL scalar count |
| **DRAM Weights energy** | All 5 | Exact match (e.g., conv4: 84,934,656 pJ) |
| **DRAM Output writes** | conv1, conv3 | 455,197 and 78,654 exact matches |
| **DRAM total energy** | conv4 | 134.25 vs 134.26 uJ = 1.00x |
| **MACs energy** | conv4 | 204.31 uJ = 1.00x (no gated compute) |
| **weights_spad fills/PE** | conv1 | 50,688/PE (validates temporal reuse fix) |

### Per-Component Energy (conv4 sparse)
| Component | AF (uJ) | SL (uJ) | Ratio |
|-----------|---------|---------|-------|
| MACs | 204.31 | 204.31 | **1.00x** |
| psum_spad | 235.86 | 236.05 | **1.00x** |
| weights_spad | 75.69 | 77.80 | **0.97x** |
| ifmap_spad | 88.15 | 93.66 | 0.94x |
| shared_glb | 300.86 | 363.96 | 0.83x |
| DRAM | 134.25 | 134.26 | **1.00x** |
| **TOTAL** | **1,039.11** | **1,110.05** | **0.94x** |

### Fixes Applied
1. **DRAM Output temporal reuse** (Fix 1): `_is_directly_above_storage()` in symbolic.py.
2. **Halo/stride fill reuse** (Fix 2): `halo_factor` in `repeat_temporal()`.
3. **Memory BW throttling** (Fix 3): `total_*`/`pu_*` latency symbols.
4. **DRAM Output sparse drain compression** (Fix 4): Child `writes_to_parent` compressed.
5. **Toll temporal reuse** (Fix 5): `_is_directly_above_storage()` skips Storage/Toll at
   components irrelevant to the tensor (e.g., ifmap_spad[Inputs] when checking Weights).
   Fixed conv4/5 DRAM Weight reads 4x inflation (N=4 below DummyBuffer Toll).
6. **Gated compute suppression** (Fix 6): Removed `gated_compute` from MACs ERT.
   SL captures gating entirely at weights_spad (gated reads), not at compute level.

### Remaining Discrepancies
- **DRAM Input reads undershoot**: conv1 AF 174,636 vs SL 776,160. Spatial multicast
  model difference — AF reuses Inputs more aggressively across spatial dims.
- **shared_glb undershoot** (conv4 0.83x): Cascading effect of DRAM Input undershoot.